# Accessing Data From The US Census

This notebook contains code to access data from the US Census' API. There are numerous datasets available ([*link*](https://www.census.gov/data/developers/data-sets.html)).

The **American Community Survey** (ACS, [*link*](https://www.census.gov/programs-surveys/acs)) contains various demographic data collected across the country. There are four versions of this survey based on the timeframe of the collected data (*1 to 5 years of data*) and the granularity of location (*city, state, ZIP Code*). The website offers guidelines for when to use which survey ([*link*](https://www.census.gov/programs-surveys/acs/guidance/estimates.html)).

The schedule for the release of the data is located [here](https://www.census.gov/programs-surveys/acs/news/data-releases/2020/release-schedule.html). As of March 18, 2022 the 2020 5-year ACS was the most recent release. Prior to the 2020 5-year ACS release the 2020 1-year ACS estimates could be used to fill in the time that the 5-year study does not cover. Furthermore, there is a 2021 Population Estimates and Projections dataset *((link)[https://www.census.gov/data/developers/data-sets/popest-popproj.html])* to supplement these surveys.

The [examples](https://api.census.gov/data/2020/acs/acs5/examples.html) page contains the formatting necessary to make calls and lists the geographic area of data that is available.

In [1]:
# Import the necessary libraries
import pandas as pd
import geopandas as gpd
import numpy as np
import requests
import json
from datetime import date
import os
from tqdm import tqdm
from sodapy import Socrata

import folium

# I. Get All Demographic Data

## I.A. Function To Get State Level Data

[5-Year ACS Main Page](https://api.census.gov/data/2020/acs/acs5.html)

Example format: https://api.census.gov/data/2020/acs/acs5?get=NAME,B01001_001E&for=state:01


You must match the zipcode with the [state code](https://api.census.gov/data/2020/acs/acs5?get=NAME&for=state:*) in order to get the data.

For instance *36* is the code for NY.

In [2]:
#A function for the API call
# https://www.w3schools.com/python/ref_requests_response.asp

# A socrata plugin alternative
# https://support.socrata.com/hc/en-us/articles/360051168614-US-Census-Gateway-Plugin

def obtain_census_data_state_5_year(year, codes, state):
    state_code_url = 'https://api.census.gov/data/{}/acs/acs5?get=NAME,{}&for=state:{}'.format(year, codes, state)
    state_code_content = requests.get(state_code_url).json()
    return state_code_content

In [3]:
# An example call for median income
obtain_census_data_state_5_year(year = '2020', codes = 'B19326_001E', state = '36')

[['NAME', 'B19326_001E', 'state'], ['New York', '35379', '36']]

The survey's variable list is [here](https://api.census.gov/data/2020/acs/acs5/variables.html), the variable codes are not always the same year to year and survey version to survey version (*i.e. the 5 year and 1 year surveys*); after review the 2020 and 2019 codes for the variables I selected are the same. There are over 66,000 variables in this dataset, loading and searching through them may take a while *([source](https://api.census.gov/data/2020/acs/acs5))*.

In [4]:
# The codes and associated definitions we will examine
census_codes = {
    "Total_Pop": "B01001_001E",
    "Total_Pop_Male": "B01001_002E",
    "Total_Pop_Female": "B01001_026E",
    "Median_Age": "B01002_001E",
    "Median_Age_Male": "B01002_002E",
    "Median_Age_Female": "B01002_003E",
}

In [5]:
inv_census_codes = {v: k for k, v in census_codes.items()}

In [6]:
# An inverted dictionary
inv_census_codes

{'B01001_001E': 'Total_Pop',
 'B01001_002E': 'Total_Pop_Male',
 'B01001_026E': 'Total_Pop_Female',
 'B01002_001E': 'Median_Age',
 'B01002_002E': 'Median_Age_Male',
 'B01002_003E': 'Median_Age_Female'}

In [7]:
#Inverts the dictionary so the columns can be renamed, add keys from the forthcoming index dictionary to this dictionary
inv_census_codes = {v: k for k, v in census_codes.items()}
# inv_census_codes.update({'District_Name':'District_Name', 'CD': 'CD', 'State_Id': 'State_Id','State': 'State', 'CD_Id_Year': 'CD_Id_Year'})

#Creates a string of codes to be used in the API call
columns_url = ''

for key in census_codes:
    columns_url += census_codes[key] + ','

# Remove the last comma
columns_url = columns_url[:-1]

## I.B. 2020 ACS Data

The call returns a list of lists, where the first list contains the column names and the subsequent lists are the rows of data. The following is a call for New York:

In [8]:
obtain_census_data_state_5_year(year = '2020', codes = columns_url, state = '36')

[['NAME',
  'B01001_001E',
  'B01001_002E',
  'B01001_026E',
  'B01002_001E',
  'B01002_002E',
  'B01002_003E',
  'state'],
 ['New York', '19514849', '9474184', '10040665', '39.0', '37.5', '40.5', '36']]

In [9]:
all_states_census_raw_2020 = obtain_census_data_state_5_year(year = '2020', codes = columns_url, state = '*')

In [10]:
# The first list is the columns
all_states_census_raw_2020[0]

['NAME',
 'B01001_001E',
 'B01001_002E',
 'B01001_026E',
 'B01002_001E',
 'B01002_002E',
 'B01002_003E',
 'state']

In [11]:
# We can reverse the codes so the Data Frame's columns will be easy to read
columns = [inv_census_codes.get(item,item)  for item in all_states_census_raw_2020[0]]

In [12]:
all_states_census_raw_2020 = pd.DataFrame(all_states_census_raw_2020[1:],columns = columns)

In [13]:
all_states_census_raw_2020.head()

,NAME,Total_Pop,Total_Pop_Male,Total_Pop_Female,Median_Age,Median_Age_Male,Median_Age_Female,state
0,Pennsylvania,12794885,6269142,6525743,40.9,39.3,42.5,42
1,California,39346023,19562882,19783141,36.7,35.6,37.9,06
2,West Virginia,1807426,893743,913683,42.7,41.3,44.1,54
3,Utah,3151239,1586950,1564289,31.1,30.4,31.8,49
4,New York,19514849,9474184,10040665,39.0,37.5,40.5,36


In [14]:
# Add suffix
all_states_census_raw_2020 = all_states_census_raw_2020.add_suffix(suffix = '_2020')
all_states_census_raw_2020.rename(columns={"NAME_2020": "NAME", "state_2020": "state"}, inplace = True)
all_states_census_raw_2020.head()

,NAME,Total_Pop_2020,Total_Pop_Male_2020,Total_Pop_Female_2020,Median_Age_2020,Median_Age_Male_2020,Median_Age_Female_2020,state
0,Pennsylvania,12794885,6269142,6525743,40.9,39.3,42.5,42
1,California,39346023,19562882,19783141,36.7,35.6,37.9,06
2,West Virginia,1807426,893743,913683,42.7,41.3,44.1,54
3,Utah,3151239,1586950,1564289,31.1,30.4,31.8,49
4,New York,19514849,9474184,10040665,39.0,37.5,40.5,36


In [15]:
all_states_census_raw_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   NAME                    52 non-null     object
 1   Total_Pop_2020          52 non-null     object
 2   Total_Pop_Male_2020     52 non-null     object
 3   Total_Pop_Female_2020   52 non-null     object
 4   Median_Age_2020         52 non-null     object
 5   Median_Age_Male_2020    52 non-null     object
 6   Median_Age_Female_2020  52 non-null     object
 7   state                   52 non-null     object
dtypes: object(8)
memory usage: 3.4+ KB


In [16]:
all_states_census_raw_2020.to_csv("data_to_plot/us_census_data/state_level_data_2020.csv", index = False)
all_states_census_raw_2020 = pd.read_csv("data_to_plot/us_census_data/state_level_data_2020.csv")

In [17]:
# Note the change in data types after the csv file is open
all_states_census_raw_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   NAME                    52 non-null     object 
 1   Total_Pop_2020          52 non-null     int64  
 2   Total_Pop_Male_2020     52 non-null     int64  
 3   Total_Pop_Female_2020   52 non-null     int64  
 4   Median_Age_2020         52 non-null     float64
 5   Median_Age_Male_2020    52 non-null     float64
 6   Median_Age_Female_2020  52 non-null     float64
 7   state                   52 non-null     int64  
dtypes: float64(3), int64(4), object(1)
memory usage: 3.4+ KB


## I.C. 2019 ACS Data

In [18]:
all_states_census_raw_2019 = obtain_census_data_state_5_year(year = '2019', codes = columns_url, state = '*')

In [19]:
# We can reverse the codes so the Data Frame's columns will be easy to read
columns = [inv_census_codes.get(item,item)  for item in all_states_census_raw_2019[0]]

In [20]:
all_states_census_raw_2019 = pd.DataFrame(all_states_census_raw_2019[1:],columns = columns)
all_states_census_raw_2019.head()

,NAME,Total_Pop,Total_Pop_Male,Total_Pop_Female,Median_Age,Median_Age_Male,Median_Age_Female,state
0,Alabama,4876250,2359355,2516895,39.0,37.5,40.5,01
1,Alaska,737068,384915,352153,34.3,33.8,34.9,02
2,Arizona,7050299,3504509,3545790,37.7,36.3,39.0,04
3,Arkansas,2999370,1471760,1527610,38.1,36.8,39.5,05
4,California,39283497,19526298,19757199,36.5,35.4,37.7,06


In [21]:
# Add suffix
all_states_census_raw_2019 = all_states_census_raw_2019.add_suffix(suffix = '_2019')
all_states_census_raw_2019.rename(columns={"NAME_2019": "NAME", "state_2019": "state"}, inplace = True)
all_states_census_raw_2019.head()

,NAME,Total_Pop_2019,Total_Pop_Male_2019,Total_Pop_Female_2019,Median_Age_2019,Median_Age_Male_2019,Median_Age_Female_2019,state
0,Alabama,4876250,2359355,2516895,39.0,37.5,40.5,01
1,Alaska,737068,384915,352153,34.3,33.8,34.9,02
2,Arizona,7050299,3504509,3545790,37.7,36.3,39.0,04
3,Arkansas,2999370,1471760,1527610,38.1,36.8,39.5,05
4,California,39283497,19526298,19757199,36.5,35.4,37.7,06


In [22]:
all_states_census_raw_2019.to_csv("data_to_plot/us_census_data/state_level_data_2019.csv", index = False)
all_states_census_raw_2019 = pd.read_csv("data_to_plot/us_census_data/state_level_data_2019.csv")

## I.C. 2021 Population Estimates and Projections Data

In [23]:
def obtain_latest_population_estimates(year, code, state):
    state_code_url = 'https://api.census.gov/data/{}/pep/population?get=NAME,{}&for=state:{}'.format(year, code, state)
    state_code_content = requests.get(state_code_url).json()
    return state_code_content

In [24]:
population_estimates_raw_2021 = obtain_latest_population_estimates(year = '2021', code = 'POP_2021', state = '*')

In [25]:
population_estimates_2021 = pd.DataFrame(population_estimates_raw_2021[1:],columns = ['NAME', 'Total_Pop_2021', 'state'])

In [26]:
population_estimates_2021.head()

,NAME,Total_Pop_2021,state
0,Oklahoma,3986639,40
1,Nebraska,1963692,31
2,Hawaii,1441553,15
3,South Dakota,895376,46
4,Tennessee,6975218,47


In [27]:
population_estimates_2021.to_csv("data_to_plot/us_census_data/state_level_data_2021.csv", index = False)
population_estimates_2021 = pd.read_csv("data_to_plot/us_census_data/state_level_data_2021.csv")

## I.D. Combine the Three Years of Data

In [28]:
all_states_census_df = all_states_census_raw_2020.copy()

In [29]:
# Add the 2019 data to the new dataframe
all_states_census_df = all_states_census_df.merge(all_states_census_raw_2019,
                                                  left_on='NAME',
                                                  right_on='NAME',
                                                  suffixes=('', '_2019'))

In [30]:
# Add the 2021 data to the new dataframe
all_states_census_df = all_states_census_df.merge(population_estimates_2021,
                                                  left_on='NAME',
                                                  right_on='NAME',
                                                  suffixes=('', '_2021'))

In [31]:
all_states_census_df.head()

,NAME,Total_Pop_2020,Total_Pop_Male_2020,Total_Pop_Female_2020,Median_Age_2020,Median_Age_Male_2020,Median_Age_Female_2020,state,Total_Pop_2019,Total_Pop_Male_2019,Total_Pop_Female_2019,Median_Age_2019,Median_Age_Male_2019,Median_Age_Female_2019,state_2019,Total_Pop_2021,state_2021
0,Pennsylvania,12794885,6269142,6525743,40.9,39.3,42.5,42,12791530,6265113,6526417,40.8,39.1,42.4,42,12964056,42
1,California,39346023,19562882,19783141,36.7,35.6,37.9,6,39283497,19526298,19757199,36.5,35.4,37.7,6,39237836,6
2,West Virginia,1807426,893743,913683,42.7,41.3,44.1,54,1817305,898578,918727,42.5,41.2,43.9,54,1782959,54
3,Utah,3151239,1586950,1564289,31.1,30.4,31.8,49,3096848,1558868,1537980,30.8,30.2,31.5,49,3337975,49
4,New York,19514849,9474184,10040665,39.0,37.5,40.5,36,19572319,9501001,10071318,38.8,37.3,40.3,36,19835913,36


In [32]:
# Drop repeat state code columns
all_states_census_df.drop(axis=1, columns=["state_2019", "state_2021"], inplace=True)

In [33]:
# Save the dataframe
all_states_census_df.to_csv("data_to_plot/us_census_data/state_level_data_2019_to_2021.csv", index = False)
all_states_census_df = pd.read_csv("data_to_plot/us_census_data/state_level_data_2019_to_2021.csv")

In [34]:
# Note the change in data types after the csv file is open
all_states_census_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   NAME                    52 non-null     object 
 1   Total_Pop_2020          52 non-null     int64  
 2   Total_Pop_Male_2020     52 non-null     int64  
 3   Total_Pop_Female_2020   52 non-null     int64  
 4   Median_Age_2020         52 non-null     float64
 5   Median_Age_Male_2020    52 non-null     float64
 6   Median_Age_Female_2020  52 non-null     float64
 7   state                   52 non-null     int64  
 8   Total_Pop_2019          52 non-null     int64  
 9   Total_Pop_Male_2019     52 non-null     int64  
 10  Total_Pop_Female_2019   52 non-null     int64  
 11  Median_Age_2019         52 non-null     float64
 12  Median_Age_Male_2019    52 non-null     float64
 13  Median_Age_Female_2019  52 non-null     float64
 14  Total_Pop_2021          52 non-null     int6

## II. Load the Geo Files

The US Census provides shapefiles

-https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html

In [35]:
usmap_gdf = gpd.read_file('data_to_plot/us_map_data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp')

In [36]:
usmap_gdf.head()

,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry
0,28,01779790,0400000US28,28,MS,Mississippi,00,121533519481,3926919758,"MULTIPOLYGON (((-88.50297 30.21523, -88.49176 ..."
1,37,01027616,0400000US37,37,NC,North Carolina,00,125923656064,13466071395,"MULTIPOLYGON (((-75.72681 35.93584, -75.71827 ..."
2,40,01102857,0400000US40,40,OK,Oklahoma,00,177662925723,3374587997,"POLYGON ((-103.00257 36.52659, -103.00219 36.6..."
3,51,01779803,0400000US51,51,VA,Virginia,00,102257717110,8528531774,"MULTIPOLYGON (((-75.74241 37.80835, -75.74151 ..."
4,54,01779805,0400000US54,54,WV,West Virginia,00,62266474513,489028543,"POLYGON ((-82.64320 38.16909, -82.64300 38.169..."


In [37]:
# Note the data types after the file is loaded
usmap_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   STATEFP   56 non-null     object  
 1   STATENS   56 non-null     object  
 2   AFFGEOID  56 non-null     object  
 3   GEOID     56 non-null     object  
 4   STUSPS    56 non-null     object  
 5   NAME      56 non-null     object  
 6   LSAD      56 non-null     object  
 7   ALAND     56 non-null     int64   
 8   AWATER    56 non-null     int64   
 9   geometry  56 non-null     geometry
dtypes: geometry(1), int64(2), object(7)
memory usage: 4.5+ KB


In [38]:
# Convert the State ID column to an integer
# From: https://stackoverflow.com/posts/60024263/revisions
usmap_gdf["GEOID"] = usmap_gdf["GEOID"].str.strip()
usmap_gdf["GEOID"] = usmap_gdf["GEOID"].astype('float')
usmap_gdf["GEOID"] = usmap_gdf["GEOID"].astype("Int64")

In [39]:
# Review the converted data types
usmap_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   STATEFP   56 non-null     object  
 1   STATENS   56 non-null     object  
 2   AFFGEOID  56 non-null     object  
 3   GEOID     56 non-null     Int64   
 4   STUSPS    56 non-null     object  
 5   NAME      56 non-null     object  
 6   LSAD      56 non-null     object  
 7   ALAND     56 non-null     int64   
 8   AWATER    56 non-null     int64   
 9   geometry  56 non-null     geometry
dtypes: Int64(1), geometry(1), int64(2), object(6)
memory usage: 4.6+ KB
